--- Day 6: Universal Orbit Map ---

You've landed at the Universal Orbit Map facility on Mercury. Because navigation in space often involves transferring between orbits, the orbit maps here are useful for finding efficient routes between, for example, you and Santa. You download a map of the local orbits (your puzzle input).

Except for the universal Center of Mass (COM), every object in space is in orbit around exactly one other object. An orbit looks roughly like this:
```
                  \
                   \
                    |
                    |
AAA--> o            o <--BBB
                    |
                    |
                   /
                  /
```
In this diagram, the object BBB is in orbit around AAA. The path that BBB takes around AAA (drawn with lines) is only partly shown. In the map data, this orbital relationship is written AAA)BBB, which means "BBB is in orbit around AAA".

Before you use your map data to plot a course, you need to make sure it wasn't corrupted during the download. To verify maps, the Universal Orbit Map facility uses orbit count checksums - the total number of direct orbits (like the one shown above) and indirect orbits.

Whenever A orbits B and B orbits C, then A indirectly orbits C. This chain can be any number of objects long: if A orbits B, B orbits C, and C orbits D, then A indirectly orbits D.

For example, suppose you have the following map:
```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L

Visually, the above map of orbits looks like this:

        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I
```
In this visual representation, when two objects are connected by a line, the one on the right directly orbits the one on the left.

Here, we can count the total number of orbits as follows:

    D directly orbits C and indirectly orbits B and COM, a total of 3 orbits.
    L directly orbits K and indirectly orbits J, E, D, C, B, and COM, a total of 7 orbits.
    COM orbits nothing.

The total number of direct and indirect orbits in this example is 42.

What is the total number of direct and indirect orbits in your map data?

In [1]:
from pathlib import Path
import itertools
from collections import Counter
from collections import defaultdict
from pprint import pprint
import re
from datetime import datetime
import numpy as np
import functools
from copy import copy, deepcopy
from collections import namedtuple

In [2]:
data_in = Path('input.txt').read_text().splitlines()

In [3]:
planets = defaultdict(list)

for orbit in data_in:
    center, orbiter = orbit.split(')')
    planets[center].append(orbiter)

In [4]:
def depth_search(planet_dict, result_dict, current, count):
    count += 1
    result_dict[current] = count
    
    if current not in planet_dict.keys():
        return result_dict
    else:
        for new_planet in planet_dict[current]:
            result_dict.update(depth_search(planet_dict, result_dict, new_planet, count))
        return result_dict

In [5]:
sum(depth_search(planets, dict(), 'COM', -1).values())

254447

--- Part Two ---

Now, you just need to figure out how many orbital transfers you (YOU) need to take to get to Santa (SAN).

You start at the object YOU are orbiting; your destination is the object SAN is orbiting. An orbital transfer lets you move from any object to an object orbiting or orbited by that object.

For example, suppose you have the following map:
```
COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN
```
Visually, the above map of orbits looks like this:
```
                          YOU
                         /
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
```
In this example, YOU are in orbit around K, and SAN is in orbit around I. To move from K to I, a minimum of 4 orbital transfers are required:
```
    K to J
    J to E
    E to D
    D to I
```
Afterward, the map of orbits looks like this:
```
        G - H       J - K - L
       /           /
COM - B - C - D - E - F
               \
                I - SAN
                 \
                  YOU
```
What is the minimum number of orbital transfers required to move from the object YOU are orbiting to the object SAN is orbiting? (Between the objects they are orbiting - not between YOU and SAN.)


In [6]:
planets_inv = dict()

for parent, children in planets.items():
    for child in children:
        planets_inv[child] = parent

In [7]:
def trace_up(target, planets_inv):
    trace = []
    current = target
    
    while current != 'COM':
        current = planets_inv[current]
        trace.append(current)

    return trace[::-1]

In [8]:
san_trace = trace_up('SAN', planets_inv)
you_trace = trace_up('YOU', planets_inv)

In [9]:
last_common = None
for s,y in zip(san_trace, you_trace):
    if s != y:
        break
    else:
        last_common = s

In [10]:
len(san_trace) - san_trace.index(last_common) - 1 + len(you_trace) - you_trace.index(last_common) - 1

445